In [1]:
# Imports

import nltk
from nltk import punkt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore
import pandas as pd
import numpy as np
import pickle

## NLP

In [2]:
with open('./Pickles/df_sneakers.pickle', 'rb') as f:
    df_sneakers = pickle.load(f)

In [3]:
df_sneakers

,name,score,bottom_line,con_nums,con_text,pro_nums,pro_text
0,Pharrell Williams x Adidas Human Race NMD,93,Adidas Originals’ NMD Human Race shoes live up...,4,[The Pharrell Williams x Adidas Human Race NMD...,8,[The Pharrell Williams x Adidas Human Race NMD...
1,Adidas Stan Smith,93,After Adidas brought back the Stan Smith about...,4,[Some of the sneakers’ diehard fans remarked t...,11,"[The main model in the Stan Smith collection, ..."
2,Adidas Continental 80,92,Adidas sneakers are known for taking inspirati...,3,"[The Adidas Continental 80 has a narrow fit, s...",10,[The majority of testers admire the Adidas Con...
3,Adidas POD-S3.1,92,The Adidas POD-S3.1 is a new model added to th...,2,[One person thought that the stitching and glu...,9,[Almost all the buyers of the Adidas POD-S3.1 ...
4,Adidas Superstar,94,The Adidas Superstar is absolutely an iconic c...,3,[Loyal Superstars wearers are disappointed tha...,9,[The majority of the reviewers like that the A...
5,Adidas Grand Court,92,The Three Stripes tweaked its tennis classics ...,2,"[This sneaker is quite narrow, according to a ...",8,[Most of the reviewers are attracted to the Ad...
6,Adidas I-5923,91,"Initially introduced as the Iniki Runner, the ...",5,"[Several, including those with wide feet, note...",9,[Many like that the Adidas I-5923 sneakers com...
7,Adidas Sobakov,92,After the release of the Adidas Kamanda with a...,5,[The upper is denser than Primeknit and isn’t ...,11,[Many hailed the design of the Adidas Sobakov ...
8,Adidas Swift Run,93,"Another low-top shoe from Adidas, the Swift Ru...",3,[Several of the users complain about the shoe ...,10,[Most of the comments state that the Adidas Sw...
9,Adidas NMD_R1 Primeknit,92,Even if the Adidas NMD_R1 Primeknit is priced ...,2,"[While they are happy with their purchase, qui...",9,[A lot of wearers love the upper of the Adidas...


In [4]:
nlp_sneakers = df_sneakers.copy()

In [5]:
# Converting lists into strings

nlp_sneakers['con_text'] = nlp_sneakers['con_text'].apply(', '.join)
nlp_sneakers['pro_text'] = nlp_sneakers['pro_text'].apply(', '.join)

In [6]:
nlp_sneakers

,name,score,bottom_line,con_nums,con_text,pro_nums,pro_text
0,Pharrell Williams x Adidas Human Race NMD,93,Adidas Originals’ NMD Human Race shoes live up...,4,The Pharrell Williams x Adidas Human Race NMD ...,8,The Pharrell Williams x Adidas Human Race NMD ...
1,Adidas Stan Smith,93,After Adidas brought back the Stan Smith about...,4,Some of the sneakers’ diehard fans remarked th...,11,"The main model in the Stan Smith collection, t..."
2,Adidas Continental 80,92,Adidas sneakers are known for taking inspirati...,3,"The Adidas Continental 80 has a narrow fit, se...",10,The majority of testers admire the Adidas Cont...
3,Adidas POD-S3.1,92,The Adidas POD-S3.1 is a new model added to th...,2,One person thought that the stitching and glui...,9,Almost all the buyers of the Adidas POD-S3.1 s...
4,Adidas Superstar,94,The Adidas Superstar is absolutely an iconic c...,3,Loyal Superstars wearers are disappointed that...,9,The majority of the reviewers like that the Ad...
5,Adidas Grand Court,92,The Three Stripes tweaked its tennis classics ...,2,"This sneaker is quite narrow, according to a c...",8,Most of the reviewers are attracted to the Adi...
6,Adidas I-5923,91,"Initially introduced as the Iniki Runner, the ...",5,"Several, including those with wide feet, note ...",9,Many like that the Adidas I-5923 sneakers come...
7,Adidas Sobakov,92,After the release of the Adidas Kamanda with a...,5,The upper is denser than Primeknit and isn’t t...,11,Many hailed the design of the Adidas Sobakov a...
8,Adidas Swift Run,93,"Another low-top shoe from Adidas, the Swift Ru...",3,Several of the users complain about the shoe h...,10,Most of the comments state that the Adidas Swi...
9,Adidas NMD_R1 Primeknit,92,Even if the Adidas NMD_R1 Primeknit is priced ...,2,"While they are happy with their purchase, quit...",9,A lot of wearers love the upper of the Adidas ...


In [7]:
# analyzer = SentimentIntensityAnalyzer()

In [8]:
# def text_analyzer(series, sentiment=None):
#     score_list = []
#     for text in series:
#         vs = analyzer.polarity_scores(text)
#         if sentiment == 'neg':
#             score_list.append(vs['neg'])
#         elif sentiment == 'neu':
#             score_list.append(vs['neu'])
#         elif sentiment == 'pos':
#             score_list.append(vs['pos'])
#         elif sentiment == 'compound':
#             score_list.append(vs['compound'])
#         else:
#             score_list.append(vs)
#     return score_list

In [9]:
# New columns for the Vader scores

# nlp_sneakers['bottom_line_score'] = text_analyzer(nlp_sneakers['bottom_line'], sentiment='compound')
# nlp_sneakers['con_text_score'] = text_analyzer(nlp_sneakers['con_text'], sentiment='neg')
# nlp_sneakers['pro_text_score'] = text_analyzer(nlp_sneakers['pro_text'], sentiment='pos')

In [10]:
# nlp_sneakers

In [11]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

In [12]:
# Removing punctuations

nlp_sneakers['bottom_line'] = nlp_sneakers['bottom_line'].apply(remove_punctuations)
nlp_sneakers['con_text'] = nlp_sneakers['con_text'].apply(remove_punctuations)
nlp_sneakers['pro_text'] = nlp_sneakers['pro_text'].apply(remove_punctuations)

In [13]:
nlp_sneakers

,name,score,bottom_line,con_nums,con_text,pro_nums,pro_text
0,Pharrell Williams x Adidas Human Race NMD,93,Adidas Originals’ NMD Human Race shoes live up...,4,The Pharrell Williams x Adidas Human Race NMD ...,8,The Pharrell Williams x Adidas Human Race NMD ...
1,Adidas Stan Smith,93,After Adidas brought back the Stan Smith about...,4,Some of the sneakers’ diehard fans remarked th...,11,The main model in the Stan Smith collection t...
2,Adidas Continental 80,92,Adidas sneakers are known for taking inspirati...,3,The Adidas Continental 80 has a narrow fit se...,10,The majority of testers admire the Adidas Cont...
3,Adidas POD-S3.1,92,The Adidas POD S3 1 is a new model added to th...,2,One person thought that the stitching and glui...,9,Almost all the buyers of the Adidas POD S3 1 s...
4,Adidas Superstar,94,The Adidas Superstar is absolutely an iconic c...,3,Loyal Superstars wearers are disappointed that...,9,The majority of the reviewers like that the Ad...
5,Adidas Grand Court,92,The Three Stripes tweaked its tennis classics ...,2,This sneaker is quite narrow according to a c...,8,Most of the reviewers are attracted to the Adi...
6,Adidas I-5923,91,Initially introduced as the Iniki Runner the ...,5,Several including those with wide feet note ...,9,Many like that the Adidas I 5923 sneakers come...
7,Adidas Sobakov,92,After the release of the Adidas Kamanda with a...,5,The upper is denser than Primeknit and isn’t t...,11,Many hailed the design of the Adidas Sobakov a...
8,Adidas Swift Run,93,Another low top shoe from Adidas the Swift Ru...,3,Several of the users complain about the shoe h...,10,Most of the comments state that the Adidas Swi...
9,Adidas NMD_R1 Primeknit,92,Even if the Adidas NMD R1 Primeknit is priced ...,2,While they are happy with their purchase quit...,9,A lot of wearers love the upper of the Adidas ...


In [14]:
# Lowercasing

nlp_sneakers['bottom_line'] = nlp_sneakers['bottom_line'].str.lower()
nlp_sneakers['con_text'] = nlp_sneakers['con_text'].str.lower()
nlp_sneakers['pro_text'] = nlp_sneakers['pro_text'].str.lower()

In [15]:
nlp_sneakers

,name,score,bottom_line,con_nums,con_text,pro_nums,pro_text
0,Pharrell Williams x Adidas Human Race NMD,93,adidas originals’ nmd human race shoes live up...,4,the pharrell williams x adidas human race nmd ...,8,the pharrell williams x adidas human race nmd ...
1,Adidas Stan Smith,93,after adidas brought back the stan smith about...,4,some of the sneakers’ diehard fans remarked th...,11,the main model in the stan smith collection t...
2,Adidas Continental 80,92,adidas sneakers are known for taking inspirati...,3,the adidas continental 80 has a narrow fit se...,10,the majority of testers admire the adidas cont...
3,Adidas POD-S3.1,92,the adidas pod s3 1 is a new model added to th...,2,one person thought that the stitching and glui...,9,almost all the buyers of the adidas pod s3 1 s...
4,Adidas Superstar,94,the adidas superstar is absolutely an iconic c...,3,loyal superstars wearers are disappointed that...,9,the majority of the reviewers like that the ad...
5,Adidas Grand Court,92,the three stripes tweaked its tennis classics ...,2,this sneaker is quite narrow according to a c...,8,most of the reviewers are attracted to the adi...
6,Adidas I-5923,91,initially introduced as the iniki runner the ...,5,several including those with wide feet note ...,9,many like that the adidas i 5923 sneakers come...
7,Adidas Sobakov,92,after the release of the adidas kamanda with a...,5,the upper is denser than primeknit and isn’t t...,11,many hailed the design of the adidas sobakov a...
8,Adidas Swift Run,93,another low top shoe from adidas the swift ru...,3,several of the users complain about the shoe h...,10,most of the comments state that the adidas swi...
9,Adidas NMD_R1 Primeknit,92,even if the adidas nmd r1 primeknit is priced ...,2,while they are happy with their purchase quit...,9,a lot of wearers love the upper of the adidas ...


In [16]:
# Removing numbers

nlp_sneakers['bottom_line'] = nlp_sneakers['bottom_line'].str.replace('\d+', ' ')
nlp_sneakers['con_text'] = nlp_sneakers['con_text'].str.replace('\d+', ' ')
nlp_sneakers['pro_text'] = nlp_sneakers['pro_text'].str.replace('\d+', ' ')

In [17]:
nlp_sneakers

,name,score,bottom_line,con_nums,con_text,pro_nums,pro_text
0,Pharrell Williams x Adidas Human Race NMD,93,adidas originals’ nmd human race shoes live up...,4,the pharrell williams x adidas human race nmd ...,8,the pharrell williams x adidas human race nmd ...
1,Adidas Stan Smith,93,after adidas brought back the stan smith about...,4,some of the sneakers’ diehard fans remarked th...,11,the main model in the stan smith collection t...
2,Adidas Continental 80,92,adidas sneakers are known for taking inspirati...,3,the adidas continental has a narrow fit sev...,10,the majority of testers admire the adidas cont...
3,Adidas POD-S3.1,92,the adidas pod s is a new model added to th...,2,one person thought that the stitching and glui...,9,almost all the buyers of the adidas pod s s...
4,Adidas Superstar,94,the adidas superstar is absolutely an iconic c...,3,loyal superstars wearers are disappointed that...,9,the majority of the reviewers like that the ad...
5,Adidas Grand Court,92,the three stripes tweaked its tennis classics ...,2,this sneaker is quite narrow according to a c...,8,most of the reviewers are attracted to the adi...
6,Adidas I-5923,91,initially introduced as the iniki runner the ...,5,several including those with wide feet note ...,9,many like that the adidas i sneakers come in...
7,Adidas Sobakov,92,after the release of the adidas kamanda with a...,5,the upper is denser than primeknit and isn’t t...,11,many hailed the design of the adidas sobakov a...
8,Adidas Swift Run,93,another low top shoe from adidas the swift ru...,3,several of the users complain about the shoe h...,10,most of the comments state that the adidas swi...
9,Adidas NMD_R1 Primeknit,92,even if the adidas nmd r primeknit is priced ...,2,while they are happy with their purchase quit...,9,a lot of wearers love the upper of the adidas ...


In [18]:
# Removing stop words

stop = set(stopwords.words('english'))
stop.add('sneaker')
stop.add('shoe')
stop.add('reviewer')
stop.add('user')
stop.add('wearer')
stop.add('buyer')
stop.add('purchaser')
stop.add('many')
stop.add('several')
stop.add('number')
stop.add('say')

nlp_sneakers['bottom_line'] = nlp_sneakers['bottom_line'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
nlp_sneakers['con_text'] = nlp_sneakers['con_text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
nlp_sneakers['pro_text'] = nlp_sneakers['pro_text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [19]:
nlp_sneakers

,name,score,bottom_line,con_nums,con_text,pro_nums,pro_text
0,Pharrell Williams x Adidas Human Race NMD,93,adidas originals’ nmd human race shoes live br...,4,pharrell williams x adidas human race nmd runs...,8,pharrell williams x adidas human race nmd look...
1,Adidas Stan Smith,93,adidas brought back stan smith years ago seen ...,4,sneakers’ diehard fans remarked leather upper ...,11,main model stan smith collection shoes comfort...
2,Adidas Continental 80,92,adidas sneakers known taking inspiration herit...,3,adidas continental narrow fit testers detected...,10,majority testers admire adidas continental ’s ...
3,Adidas POD-S3.1,92,adidas pod new model added adidas line making ...,2,one person thought stitching gluing knit onto ...,9,almost buyers adidas pod shoes said would reco...
4,Adidas Superstar,94,adidas superstar absolutely iconic classic lov...,3,loyal superstars wearers disappointed quality ...,9,majority reviewers like adidas superstar iconi...
5,Adidas Grand Court,92,three stripes tweaked tennis classics fresher ...,2,quite narrow according couple wearers users fe...,8,reviewers attracted adidas grand court’s simpl...
6,Adidas I-5923,91,initially introduced iniki runner low top unkn...,5,including wide feet note fit extensive usual a...,9,like adidas sneakers come variety fresh colorw...
7,Adidas Sobakov,92,release adidas kamanda deliberately misshapen ...,5,upper denser primeknit isn’t stretchy knitted ...,11,hailed design adidas sobakov innovative unique...
8,Adidas Swift Run,93,another low top adidas swift run offers great ...,3,users complain poor quality couple users point...,10,comments state adidas swift run comfortable go...
9,Adidas NMD_R1 Primeknit,92,even adidas nmd r primeknit priced average rat...,2,happy purchase quite buyers still think pair s...,9,lot wearers love upper adidas nmd r primeknit ...


In [20]:
# Lemmatizing verbs

lemmatizer = WordNetLemmatizer()

nlp_sneakers['bottom_line'] = nlp_sneakers['bottom_line'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
nlp_sneakers['con_text'] = nlp_sneakers['con_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
nlp_sneakers['pro_text'] = nlp_sneakers['pro_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [21]:
nlp_sneakers

,name,score,bottom_line,con_nums,con_text,pro_nums,pro_text
0,Pharrell Williams x Adidas Human Race NMD,93,adidas originals’ nmd human race shoe live bra...,4,pharrell williams x adidas human race nmd run ...,8,pharrell williams x adidas human race nmd look...
1,Adidas Stan Smith,93,adidas brought back stan smith year ago seen r...,4,sneakers’ diehard fan remarked leather upper l...,11,main model stan smith collection shoe comforta...
2,Adidas Continental 80,92,adidas sneaker known taking inspiration herita...,3,adidas continental narrow fit tester detected ...,10,majority tester admire adidas continental ’s u...
3,Adidas POD-S3.1,92,adidas pod new model added adidas line making ...,2,one person thought stitching gluing knit onto ...,9,almost buyer adidas pod shoe said would recomm...
4,Adidas Superstar,94,adidas superstar absolutely iconic classic lov...,3,loyal superstar wearer disappointed quality se...,9,majority reviewer like adidas superstar iconic...
5,Adidas Grand Court,92,three stripe tweaked tennis classic fresher cl...,2,quite narrow according couple wearer user feel...,8,reviewer attracted adidas grand court’s simpli...
6,Adidas I-5923,91,initially introduced iniki runner low top unkn...,5,including wide foot note fit extensive usual a...,9,like adidas sneaker come variety fresh colorwa...
7,Adidas Sobakov,92,release adidas kamanda deliberately misshapen ...,5,upper denser primeknit isn’t stretchy knitted ...,11,hailed design adidas sobakov innovative unique...
8,Adidas Swift Run,93,another low top adidas swift run offer great c...,3,user complain poor quality couple user point k...,10,comment state adidas swift run comfortable goo...
9,Adidas NMD_R1 Primeknit,92,even adidas nmd r primeknit priced average rat...,2,happy purchase quite buyer still think pair sh...,9,lot wearer love upper adidas nmd r primeknit h...


In [22]:
# Tokenizing

tokenizer = word_tokenize

nlp_sneakers['bottom_line'] = nlp_sneakers['bottom_line'].apply(tokenizer)
nlp_sneakers['con_text'] = nlp_sneakers['con_text'].apply(tokenizer)
nlp_sneakers['pro_text'] = nlp_sneakers['pro_text'].apply(tokenizer)

In [23]:
nlp_sneakers

,name,score,bottom_line,con_nums,con_text,pro_nums,pro_text
0,Pharrell Williams x Adidas Human Race NMD,93,"[adidas, originals, ’, nmd, human, race, shoe,...",4,"[pharrell, williams, x, adidas, human, race, n...",8,"[pharrell, williams, x, adidas, human, race, n..."
1,Adidas Stan Smith,93,"[adidas, brought, back, stan, smith, year, ago...",4,"[sneakers, ’, diehard, fan, remarked, leather,...",11,"[main, model, stan, smith, collection, shoe, c..."
2,Adidas Continental 80,92,"[adidas, sneaker, known, taking, inspiration, ...",3,"[adidas, continental, narrow, fit, tester, det...",10,"[majority, tester, admire, adidas, continental..."
3,Adidas POD-S3.1,92,"[adidas, pod, new, model, added, adidas, line,...",2,"[one, person, thought, stitching, gluing, knit...",9,"[almost, buyer, adidas, pod, shoe, said, would..."
4,Adidas Superstar,94,"[adidas, superstar, absolutely, iconic, classi...",3,"[loyal, superstar, wearer, disappointed, quali...",9,"[majority, reviewer, like, adidas, superstar, ..."
5,Adidas Grand Court,92,"[three, stripe, tweaked, tennis, classic, fres...",2,"[quite, narrow, according, couple, wearer, use...",8,"[reviewer, attracted, adidas, grand, court, ’,..."
6,Adidas I-5923,91,"[initially, introduced, iniki, runner, low, to...",5,"[including, wide, foot, note, fit, extensive, ...",9,"[like, adidas, sneaker, come, variety, fresh, ..."
7,Adidas Sobakov,92,"[release, adidas, kamanda, deliberately, missh...",5,"[upper, denser, primeknit, isn, ’, t, stretchy...",11,"[hailed, design, adidas, sobakov, innovative, ..."
8,Adidas Swift Run,93,"[another, low, top, adidas, swift, run, offer,...",3,"[user, complain, poor, quality, couple, user, ...",10,"[comment, state, adidas, swift, run, comfortab..."
9,Adidas NMD_R1 Primeknit,92,"[even, adidas, nmd, r, primeknit, priced, aver...",2,"[happy, purchase, quite, buyer, still, think, ...",9,"[lot, wearer, love, upper, adidas, nmd, r, pri..."


In [24]:
# pwd

In [25]:
# with open('/Users/flatironschool/Flatiron_Projects/Adike/Pickles/nlp_sneakers.pickle', 'wb') as f:
#         pickle.dump(nlp_sneakers, f, pickle.HIGHEST_PROTOCOL)

## LDA

In [26]:
df_bottom_line = pd.DataFrame(nlp_sneakers['bottom_line'])

In [27]:
df_con_text = pd.DataFrame(nlp_sneakers['con_text'])

In [28]:
df_pro_text = pd.DataFrame(nlp_sneakers['pro_text'])

In [29]:
def get_words(df):
    all_words = []
    for i in range(0, 844):
        for s in df.loc[i][0]:
            if s == 're':
                df.loc[i][0].remove(s)
            df.loc[i][0] = [s for s in df.loc[i][0] if len(s) > 1]
        all_words.append(df.loc[i][0])
    return all_words

In [30]:
list_bottom_line = get_words(df_bottom_line)

In [31]:
list_con_text = get_words(df_con_text)

In [32]:
list_pro_text = get_words(df_pro_text)

In [33]:
dictionary_b = Dictionary(list_bottom_line)

In [34]:
dictionary_c = Dictionary(list_con_text)

In [35]:
dictionary_p = Dictionary(list_pro_text)

In [36]:
bow_corpus_b = [dictionary_b.doc2bow(doc) for doc in list_bottom_line]

In [37]:
bow_corpus_c = [dictionary_c.doc2bow(doc) for doc in list_con_text]

In [38]:
bow_corpus_p = [dictionary_p.doc2bow(doc) for doc in list_pro_text]

In [39]:
lda_b = LdaMulticore(bow_corpus_b, id2word=dictionary_b, num_topics=10, random_state = 22, passes=10, per_word_topics=True)

In [40]:
lda_b.print_topics()

[(0,
  '0.017*"nike" + 0.009*"adidas" + 0.008*"silhouette" + 0.007*"upper" + 0.006*"comfortable" + 0.005*"comfort" + 0.005*"two" + 0.005*"sneaker" + 0.005*"flyknit" + 0.005*"air"'),
 (1,
  '0.021*"adidas" + 0.012*"look" + 0.009*"feel" + 0.008*"comfort" + 0.008*"silhouette" + 0.007*"nike" + 0.007*"classic" + 0.007*"also" + 0.007*"style" + 0.006*"low"'),
 (2,
  '0.023*"adidas" + 0.014*"superstar" + 0.013*"comfort" + 0.010*"classic" + 0.010*"price" + 0.008*"pair" + 0.008*"design" + 0.007*"silhouette" + 0.007*"style" + 0.007*"stylish"'),
 (3,
  '0.018*"nike" + 0.013*"air" + 0.012*"design" + 0.011*"comfort" + 0.010*"adidas" + 0.010*"shoe" + 0.008*"classic" + 0.007*"well" + 0.007*"kick" + 0.007*"style"'),
 (4,
  '0.017*"nike" + 0.009*"comfort" + 0.007*"high" + 0.007*"design" + 0.006*"cushioning" + 0.006*"air" + 0.006*"top" + 0.005*"version" + 0.005*"excellent" + 0.005*"support"'),
 (5,
  '0.020*"nike" + 0.011*"adidas" + 0.010*"style" + 0.008*"comfortable" + 0.008*"price" + 0.008*"look" + 0.0

In [41]:
lda_c = LdaMulticore(bow_corpus_c, id2word=dictionary_c, num_topics=10, random_state = 22, passes=10, per_word_topics=True)

In [42]:
lda_c.print_topics()

[(0,
  '0.019*"nike" + 0.019*"buyer" + 0.014*"one" + 0.014*"shoe" + 0.012*"air" + 0.012*"couple" + 0.012*"complained" + 0.011*"feel" + 0.010*"user" + 0.010*"narrow"'),
 (1,
  '0.024*"adidas" + 0.022*"one" + 0.018*"narrow" + 0.018*"run" + 0.018*"reviewer" + 0.015*"bit" + 0.014*"foot" + 0.013*"buyer" + 0.013*"nike" + 0.012*"shoe"'),
 (2,
  '0.030*"adidas" + 0.015*"buyer" + 0.015*"upper" + 0.013*"support" + 0.012*"one" + 0.012*"primeknit" + 0.012*"material" + 0.011*"would" + 0.010*"complained" + 0.010*"heel"'),
 (3,
  '0.024*"adidas" + 0.023*"one" + 0.023*"user" + 0.019*"reviewer" + 0.017*"nike" + 0.014*"pair" + 0.013*"narrow" + 0.012*"couple" + 0.011*"get" + 0.011*"price"'),
 (4,
  '0.017*"upper" + 0.016*"reviewer" + 0.014*"adidas" + 0.014*"one" + 0.012*"user" + 0.012*"easily" + 0.012*"shoe" + 0.012*"bit" + 0.012*"material" + 0.011*"narrow"'),
 (5,
  '0.033*"one" + 0.020*"nike" + 0.019*"air" + 0.018*"reviewer" + 0.015*"couple" + 0.014*"said" + 0.014*"bit" + 0.013*"user" + 0.011*"narrow" 

In [43]:
lda_p = LdaMulticore(bow_corpus_p, id2word=dictionary_p, num_topics=10, random_state = 22, passes=10, per_word_topics=True)

In [44]:
lda_p.print_topics()

[(0,
  '0.031*"nike" + 0.017*"reviewer" + 0.017*"user" + 0.014*"foot" + 0.014*"air" + 0.014*"buyer" + 0.012*"according" + 0.012*"one" + 0.011*"wearer" + 0.008*"look"'),
 (1,
  '0.018*"nike" + 0.014*"user" + 0.011*"high" + 0.010*"top" + 0.009*"low" + 0.009*"feel" + 0.008*"comfortable" + 0.008*"look" + 0.008*"good" + 0.007*"casual"'),
 (2,
  '0.040*"adidas" + 0.020*"reviewer" + 0.020*"one" + 0.016*"buyer" + 0.011*"shoe" + 0.011*"comfortable" + 0.011*"said" + 0.010*"user" + 0.008*"according" + 0.008*"superstar"'),
 (3,
  '0.033*"flyknit" + 0.027*"nike" + 0.012*"reviewer" + 0.010*"upper" + 0.009*"air" + 0.008*"shoe" + 0.007*"huarache" + 0.007*"shox" + 0.006*"user" + 0.006*"design"'),
 (4,
  '0.024*"nike" + 0.024*"air" + 0.019*"shoe" + 0.018*"reviewer" + 0.012*"max" + 0.012*"comfortable" + 0.011*"good" + 0.009*"user" + 0.009*"purchaser" + 0.009*"design"'),
 (5,
  '0.046*"adidas" + 0.014*"user" + 0.012*"kick" + 0.011*"reviewer" + 0.010*"comfortable" + 0.010*"sneaker" + 0.009*"purchaser" + 0.